notebook to label GSO shoe meshes with 3 keypoints on the nnose, heel and top of the heel.


determines principal axis using PCA and assumes the shoe  is oriented upright. heel is the side for which the Z-values are larger (heuristic)

In [38]:
import open3d as o3d
import trimesh

In [39]:
# load the meshes
import numpy as np
# mesh path
from dsd import DATA_DIR

mug_mesh_path = DATA_DIR / "meshes" / "shoes" / "GSO"
meshes = list(mug_mesh_path.glob("**/*.obj"))
print(meshes)

[PosixPath('/home/tlips/Code/diffusing-synthetic-data/data/meshes/shoes/GSO/Lovestruck_Tieks_Glittery_Rose_Gold_Italian_Leather_Ballet_Flats.obj'), PosixPath('/home/tlips/Code/diffusing-synthetic-data/data/meshes/shoes/GSO/Mens_Gold_Cup_ASV_Dress_Casual_Venetian_in_Dark_Brown_Leather.obj'), PosixPath('/home/tlips/Code/diffusing-synthetic-data/data/meshes/shoes/GSO/ENFR_MID_ENFORCER.obj'), PosixPath('/home/tlips/Code/diffusing-synthetic-data/data/meshes/shoes/GSO/Rose_Garden_Tieks_Leather_Ballet_Flats_with_Floral_Rosettes.obj'), PosixPath('/home/tlips/Code/diffusing-synthetic-data/data/meshes/shoes/GSO/Reebok_ZIGTECH_SHARK_MAYHEM360.obj'), PosixPath('/home/tlips/Code/diffusing-synthetic-data/data/meshes/shoes/GSO/Womens_Hikerfish_Boot_in_Black_Leopard_ridcCWsv8rW.obj'), PosixPath('/home/tlips/Code/diffusing-synthetic-data/data/meshes/shoes/GSO/FYW_ALTERNATION.obj'), PosixPath('/home/tlips/Code/diffusing-synthetic-data/data/meshes/shoes/GSO/F5_TRX_FG.obj'), PosixPath('/home/tlips/Code/di

In [40]:
# load first mesh
i = 193
mesh = o3d.io.read_triangle_mesh(str(meshes[i]))

# simplify the mesh
mesh = mesh.simplify_quadric_decimation(4000)


#### get the principal axis of the z-projected mesh

vertices = np.array(mesh.vertices)
xy_vertices = vertices[:, :2]
# do pca on the xy vertices
cov = np.cov(xy_vertices.T)
eig_vals, eig_vecs = np.linalg.eig(cov)
# get the principal axis
principal_xy_axis = eig_vecs[:, np.argmax(eig_vals)]

# visualize principal axis as line 
line = o3d.geometry.LineSet()
mean = np.mean(xy_vertices, axis=0)
helper = mean + principal_xy_axis
# make 3d points
mean = np.append(mean, 0)
helper = np.append(helper, 0)
line.points = o3d.utility.Vector3dVector([mean, helper])
line.lines = o3d.utility.Vector2iVector([[0, 1]])





# find the outer points on that line in the mesh

# project the vertices on the principal axis
principal_axis = np.append(principal_xy_axis, 0)

# only take vertices which are close to the principal axis
vertices = np.array(mesh.vertices)
vertices_2d = vertices[:, :2]
vertices_2d = vertices_2d/ np.linalg.norm(vertices_2d, axis=1)[:, None]
projected_vertices = np.dot(vertices_2d, principal_xy_axis) / np.linalg.norm(principal_xy_axis) / np.linalg.norm(vertices_2d, axis=1)
# get vertices with angle less than 5 degrees
vertices = vertices[np.where(np.abs(projected_vertices) > 0.999)] # 3 degrees

# visualize these vertices
vertices_o3d = o3d.geometry.PointCloud()
vertices_o3d.points = o3d.utility.Vector3dVector(vertices)
# o3d.visualization.draw_geometries([mesh, vertices_o3d, line])



z_selected_vertices = vertices[np.where(vertices[:, 2] < 0.05)]
projected_vertices = np.dot(z_selected_vertices, principal_axis) / np.linalg.norm(principal_axis)
front_idx,back_idx = np.argmax(projected_vertices), np.argmin(projected_vertices)
front_point, back_point = z_selected_vertices[front_idx], z_selected_vertices[back_idx]



### determine which side of the shoe is the heel
# proxy: the zone around the two extreme points will have larger max z value for the heel


back_zone_vertices = vertices[np.where(np.linalg.norm(vertices[...,:2] - back_point[:2], axis=1) < 0.05)]
front_zone_vertices = vertices[np.where(np.linalg.norm(vertices[...,:2] - front_point[:2], axis=1) < 0.05)]

# # visualize the back and front zone
back_zone_o3d = o3d.geometry.PointCloud()
back_zone_o3d.points = o3d.utility.Vector3dVector(back_zone_vertices)
back_zone_o3d.paint_uniform_color([1, 0, 0])

front_zone_o3d = o3d.geometry.PointCloud()
front_zone_o3d.points = o3d.utility.Vector3dVector(front_zone_vertices)
front_zone_o3d.paint_uniform_color([0, 1, 0])

o3d.visualization.draw_geometries([mesh, back_zone_o3d, front_zone_o3d, line])
print(np.max(back_zone_vertices[:, 2]), np.max(front_zone_vertices[:, 2]))

if np.max(back_zone_vertices[:, 2]) < np.max(front_zone_vertices[:, 2]):
    print("heel is the front point")
    # heel is the front point 
    heel_point = front_point
    nose_point = back_point
    heel_zone_vertices = front_zone_vertices
    toe_zone_vertices = back_zone_vertices
else:
    heel_point = back_point
    nose_point = front_point
    heel_zone_vertices = back_zone_vertices
    toe_zone_vertices = front_zone_vertices




# get the top heel point
top_index = np.argmax(heel_zone_vertices[:, 2])
top_point = heel_zone_vertices[top_index]




# visualize the min and max points
heel_point_o3d = o3d.geometry.TriangleMesh.create_sphere(radius=0.01)
heel_point_o3d.translate(heel_point)
heel_point_o3d.paint_uniform_color([1, 0, 0])

toe_point_o3d = o3d.geometry.TriangleMesh.create_sphere(radius=0.01)
toe_point_o3d.translate(nose_point)
toe_point_o3d.paint_uniform_color([0, 1, 0])

top_point_o3d = o3d.geometry.TriangleMesh.create_sphere(radius=0.01)
top_point_o3d.translate(top_point)
top_point_o3d.paint_uniform_color([0, 0, 1])



# visualize the mesh
o3d.visualization.draw_geometries([mesh, line, heel_point_o3d, toe_point_o3d, top_point_o3d,vertices_o3d])

[Open3D WARNING] [SimplifyQuadricDecimation] This mesh contains triangle uvs that are not handled in this function
0.088042501360178 0.04808599874377251


In [41]:
def get_shoe_keypoints(vertices):


    #### get the principal axis of the z-projected mesh

    xy_vertices = vertices[:, :2]
    # do pca on the xy vertices
    cov = np.cov(xy_vertices.T)
    eig_vals, eig_vecs = np.linalg.eig(cov)
    # get the principal axis
    principal_xy_axis = eig_vecs[:, np.argmax(eig_vals)]

    # visualize principal axis as line 
    line = o3d.geometry.LineSet()
    mean = np.mean(xy_vertices, axis=0)
    helper = mean + principal_xy_axis
    # make 3d points
    mean = np.append(mean, 0)
    helper = np.append(helper, 0)
    line.points = o3d.utility.Vector3dVector([mean, helper])
    line.lines = o3d.utility.Vector2iVector([[0, 1]])





    # find the outer points on that line in the mesh

    # project the vertices on the principal axis
    principal_axis = np.append(principal_xy_axis, 0)

    # only take vertices which are close to the principal axis
    vertices_2d = vertices[:, :2]
    vertices_2d = vertices_2d/ np.linalg.norm(vertices_2d, axis=1)[:, None]
    projected_vertices = np.dot(vertices_2d, principal_xy_axis) / np.linalg.norm(principal_xy_axis) / np.linalg.norm(vertices_2d, axis=1)
    # get vertices with angle less than 3 degrees
    vertices = vertices[np.where(np.abs(projected_vertices) > 0.999)] # 3 degrees

    # visualize these vertices
    vertices_o3d = o3d.geometry.PointCloud()
    vertices_o3d.points = o3d.utility.Vector3dVector(vertices)
    # o3d.visualization.draw_geometries([mesh, vertices_o3d, line])



    z_selected_vertices = vertices[np.where(vertices[:, 2] < 0.05)]
    projected_vertices = np.dot(z_selected_vertices, principal_axis) / np.linalg.norm(principal_axis)
    front_idx,back_idx = np.argmax(projected_vertices), np.argmin(projected_vertices)
    front_point, back_point = z_selected_vertices[front_idx], z_selected_vertices[back_idx]



    ### determine which side of the shoe is the heel
    # proxy: the zone around the two extreme points will have larger max z value for the heel


    back_zone_vertices = vertices[np.where(np.linalg.norm(vertices[...,:2] - back_point[:2], axis=1) < 0.05)]
    front_zone_vertices = vertices[np.where(np.linalg.norm(vertices[...,:2] - front_point[:2], axis=1) < 0.05)]

    if np.max(back_zone_vertices[:, 2]) < np.max(front_zone_vertices[:, 2]):
        # heel is the front point 
        heel_point = front_point
        nose_point = back_point
        heel_zone_vertices = front_zone_vertices
    else:
        heel_point = back_point
        nose_point = front_point
        heel_zone_vertices = back_zone_vertices


    # get the top heel point
    top_index = np.argmax(heel_zone_vertices[:, 2])
    top_point = heel_zone_vertices[top_index]


    return {"heel": heel_point.tolist(), "nose": nose_point.tolist(), "top": top_point.tolist()}



In [ ]:
TARGET_DIR = DATA_DIR / "meshes" /"shoes" / "GSO-labeled"
TARGET_DIR.mkdir(exist_ok=True, parents=True)
import pathlib
import shutil 
import tqdm


for mesh_path in tqdm.tqdm(meshes):

    if "flip" in str(mesh_path).lower():
        continue
    if "ballet" in str(mesh_path).lower():
        continue

    # ignore flip-flops
    if "santa_cruz" in str(mesh_path).lower() or "sandal" in str(mesh_path).lower() or "adizero" in str(mesh_path).lower() or "reef" in str(mesh_path).lower() or"flip" in str(mesh_path).lower() or "chelsea" in str(mesh_path).lower():
        continue
    mesh = o3d.io.read_triangle_mesh(str(mesh_path))
    vertices = np.array(mesh.vertices)
    keypoints = get_shoe_keypoints(vertices)

    mesh_stem = pathlib.Path(str(mesh_path)).stem

    # copy mesh to target dir
    target_mesh_path = TARGET_DIR / f"{mesh_stem}.obj"
    shutil.copy(mesh_path, target_mesh_path)

    # save keypoints as json
    import json
    keypoints_path = TARGET_DIR / f"{mesh_stem}_keypoints.json"
    with open(keypoints_path, "w") as f:
        json.dump(keypoints, f)
    



100%|██████████| 255/255 [00:30<00:00,  8.29it/s]
